In [1]:
import argparse

import torch
from scipy.linalg import inv
from src.utils import *
from src.WLNodeColoring import *
from src.GraphBatching import *
import pickle

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
file_dir = 'data/DBP15K/zh_en/'
lang_num = 2

In [4]:
def load_nodes(file_dir, lang_num):
    paths = [file_dir + "/ent_ids_" + str(i) for i in range(1, lang_num + 1)]
    node_sets = []
    for path in paths:
        node_set = np.genfromtxt(path, dtype=np.int32)[:, 0]
        node_sets.append(node_set)
    return node_sets

In [5]:
def load_links(file_dir, lang_num):
    paths = [file_dir + "/triples_" + str(i) for i in range(1, lang_num + 1)]
    link_sets = []
    pred_sets = []
    for path in paths:
        triples = np.genfromtxt(path, dtype=np.int32)
        link_set = triples[:, [0, 2]]
        pred_set = triples[:, 1]
        link_sets.append(link_set)
        pred_sets.append(pred_set)
    return link_sets, pred_sets

In [6]:
def load_data(file_dir, lang_num):
    node_sets = load_nodes(file_dir, lang_num)
    link_sets, pred_sets = load_links(file_dir, lang_num)
    all_links = np.concatenate(link_sets)
    all_nodes = np.concatenate(node_sets)

    num_links = all_links.shape[0]
    num_nodes = all_nodes.shape[0]
    adj_matrix = sp.coo_matrix((np.ones(num_links * 2),
                                (np.concatenate([all_links[:, 0], all_links[:, 1]]),
                                 np.concatenate([all_links[:, 1], all_links[:, 0]]))),
                               shape=(num_nodes, num_nodes), dtype=np.float32)
    return all_links, all_nodes, adj_matrix

In [7]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.FloatTensor(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [8]:
all_links, all_nodes, adj = load_data(file_dir, lang_num)
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
norm_adj = adj_normalize(adj + sp.eye(adj.shape[0]))

In [25]:
eigen_adj_before_inv = torch.tensor((sp.eye(adj.shape[0]) - (1 - c) * norm_adj).todense())
eigen_adj_before_inv = eigen_adj_before_inv.to(device)

In [ ]:
c = 0.15
eigen_adj = c * torch.inverse(eigen_adj_before_inv)

In [20]:
eigen_adj_before_inv

tensor(indices=tensor([[    0,   403,  1559,  ..., 14282, 36612, 38959],
                       [    0,     0,     0,  ..., 38959, 38959, 38959]]),
       values=tensor([ 0.9150, -0.2082, -0.0810,  ..., -0.0911, -0.0927,
                       0.7167]),
       device='cuda:0', size=(38960, 38960), nnz=298096, layout=torch.sparse_coo)

In [9]:
wl_node_coloring = WLNodeColoring(all_nodes, all_links)
wl = wl_node_coloring.run()

In [29]:
a = norm_adj.todense()

In [31]:
a = norm_adj[:3, :3]

In [18]:
a = sparse_mx_to_torch_sparse_tensor(a)

In [22]:
a = a.to(device)

In [26]:
a = torch.inverse(a.to_dense())

In [28]:
a

tensor([[10.,  0.,  0.],
        [ 0., 10.,  0.],
        [ 0.,  0.,  4.]], device='cuda:0')

In [34]:
inv(a.todense())

array([[10.,  0., -0.],
       [ 0., 10., -0.],
       [ 0.,  0.,  4.]])